# COMPUTATION OF MSE BETWEEN ATLAS AND CANOPY DATA

<div class="alert alert-block alert-warning">
    Purpose: Compute the Mean Squared Error (MSE) between the heights in the Halle urban atlas 2012 data layer and heights estimated as the difference between DEM and DTM.
</div>

In [1]:
import enum
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
import rasterio

In [2]:
def openRaster(fn, access=0):
    ds = gdal.Open(fn, access)
    if ds is None:
        print("Error opening raster dataset")
    return ds
    
def getRasterBand(fn, band=1):
    ds = openRaster(fn)
    band = ds.GetRasterBand(1).ReadAsArray()
    return band
    
def createRasterFromCopy(fn, ds, data, driverFmt="GTiff"):
    driver = gdal.GetDriverByName(driverFmt)
    outds = driver.CreateCopy(fn, ds, strict=0)
    outds.GetRasterBand(1).WriteArray(data)
    ds=None
    outds=None

def plot(ds, title):
    fig, ax = plt.subplots(figsize=(12, 10))
    ds.plot(ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Longitude [deg]')
    ax.set_ylabel('Latitude [deg]')
    
class GdalDTypes(enum.Enum):
    """ Class to map GDAL data types to numpy"""
    uint8 = 1
    int8 = 1
    uint16 = 2
    int16 = 3
    uint32 = 4
    int32 = 5
    float32 = 6
    float64 = 7
    complex64 =  10
    complex128 = 11
    
# TODO: Make a dataclass    
def print_raster_info(ds, name=None):
    if name: 
        print(name)
    print(f'shape: {raster_shape(ds)}')
    print(f'dtype: {raster_dtype(ds)}\n')    
    
def raster_shape(ds):
    return (ds.RasterCount, ds.RasterXSize, ds.RasterYSize)

def raster_dtype(ds):
    return GdalDTypes(ds.GetRasterBand(1).DataType).name

In [3]:
atlasimgpath = 'halle_urban_atlas_2012.tiff'
canimgpath = 'dem_dtm_heights.tiff'

In [4]:
atlasds = gdal.Open(atlasimgpath)
canpyds = gdal.Open(canimgpath)

## Basic layer info

In [5]:
print_raster_info(atlasds, 'Atlas')

Atlas
shape: (1, 1575, 1466)
dtype: uint16



In [6]:
print_raster_info(canpyds, 'DEM-DTM')

DEM-DTM
shape: (1, 1202, 1011)
dtype: float64



<font color=red> **REMARK:** The dtypes and shapes of the two raster layers are different</font>

## Data type conversion

In [7]:
dstfile = "uint16_dem_dtm_heights.tiff"

In [8]:
type(canpyds)

osgeo.gdal.Dataset

In [9]:
srcdata = canpyds.ReadAsArray()

In [10]:
srcdata.dtype

dtype('float64')

In [11]:
dstdata = srcdata.astype('uint16')

In [12]:
dstdata

array([[65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       ...,
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535]], dtype=uint16)

In [13]:
dstdata.min(), dstdata.max()

(0, 65535)

In [14]:
# Get canopy parameters to be transfered to the new data type
geotransform = canpyds.GetGeoTransform()
spatialreference = canpyds.GetProjection()
nband, ncol, nrow = raster_shape(canpyds)

In [15]:
# Parameters for output dataset
driver_name = 'GTiff'
driver = gdal.GetDriverByName(driver_name)
dstds = driver.Create(dstfile, ncol, nrow, nband, gdal.GDT_UInt16)
dstds.SetGeoTransform(geotransform)
dstds.SetProjection(spatialreference)
dstds.GetRasterBand(1).WriteArray(dstdata)
dstds = None

In [16]:
uint16cands = gdal.Open(dstfile)

In [17]:
print_raster_info(uint16cands)

shape: (1, 1202, 1011)
dtype: uint16



## Clipping of atlas tiff

In [18]:
geoTransform = canpyds.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * canpyds.RasterXSize
miny = maxy + geoTransform[5] * canpyds.RasterYSize
print([minx, miny, maxx, maxy])
# canpyds = None

[4451610.0, 3148570.0, 4463630.0, 3158680.0]


In [19]:
[minx, miny, maxx, maxy]

[4451610.0, 3148570.0, 4463630.0, 3158680.0]

In [20]:
# Hrizontal and vertical resolutions
xRes = geoTransform[1]
yRes = -geoTransform[5]

In [21]:
xRes, yRes

(10.0, 10.0)

In [22]:
gdal.Translate?

Signature: gdal.Translate(destName, srcDS, **kwargs)
Docstring:
Convert a dataset.
Arguments are :
  destName --- Output dataset name
  srcDS --- a Dataset object or a filename
Keyword arguments are :
  options --- return of gdal.TranslateOptions(), string or array of strings
  other keywords arguments of gdal.TranslateOptions()
If options is provided as a gdal.TranslateOptions() object, other keywords are ignored. 
File:      ~/.local/lib/python3.10/site-packages/osgeo/gdal.py
Type:      function

In [23]:
gdal.TranslateOptions?

Signature:
gdal.TranslateOptions(
    options=None,
    format=None,
    outputType=0,
    bandList=None,
    maskBand=None,
    width=0,
    height=0,
    widthPct=0.0,
    heightPct=0.0,
    xRes=0.0,
    yRes=0.0,
    creationOptions=None,
    srcWin=None,
    projWin=None,
    projWinSRS=None,
    strict=False,
    unscale=False,
    scaleParams=None,
    exponents=None,
    outputBounds=None,
    metadataOptions=None,
    outputSRS=None,
    nogcp=False,
    GCPs=None,
    noData=None,
    rgbExpand=None,
    stats=False,
    rat=True,
    resampleAlg=None,
    callback=None,
    callback_data=None,
)
Docstring:
Create a TranslateOptions() object that can be passed to gdal.Translate()
Keyword arguments are :
  options --- can be be an array of strings, a string or let empty and filled from other keywords.
  format --- output format ("GTiff", etc...)
  outputType --- output type (gdalconst.GDT_Byte, etc...)
  bandList --- array of band numbers (index start at 1)
  maskBand --- mask

In [24]:
print_raster_info(atlasds, 'Atlas')

Atlas
shape: (1, 1575, 1466)
dtype: uint16



In [25]:
print_raster_info(canpyds, 'DEM-DTM')

DEM-DTM
shape: (1, 1202, 1011)
dtype: float64



**<font color=red>NOTE: </font>** <font color=red>The right order of coordinates to assign to the input image is: **ulx uly lrx lry**, i.e. upper-left x and y, lower-right x and y, as reported in gdal_translate documentation</font>

In [26]:
clipped_halle_urban_atlas_2012 = gdal.Translate(srcDS=atlasds, destName="clipped_halle_urban_atlas_2012.tiff", projWin=[minx, maxy, maxx, miny], outputType=gdal.GDT_UInt16, format="GTiff", noData=65535)

In [27]:
print_raster_info(clipped_halle_urban_atlas_2012, 'Clipped atlas')

Clipped atlas
shape: (1, 1202, 1011)
dtype: uint16



## Creation of binary mask

In [28]:
# Get the array data of each dataset
atlasdata = clipped_halle_urban_atlas_2012.ReadAsArray()
demdtmdata = uint16cands.ReadAsArray()

In [29]:
atlasdata

array([[65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       ...,
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535]], dtype=uint16)

In [30]:
demdtmdata

array([[65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       ...,
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535]], dtype=uint16)

In [31]:
# Create masks for valid values
atlasmasked = np.ma.masked_where(65535, atlasdata)
demdtmmasked = np.ma.masked_where(65535, demdtmdata)

In [32]:
atlasmasked

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=999999,
  dtype=uint16)

In [33]:
demdtmmasked

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=999999,
  dtype=uint16)

In [34]:
# Find the overlap of the two masked arrays
overlapmask = np.bitwise_and(atlasmasked.mask, demdtmmasked.mask)

In [35]:
overlapmask

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

## Compute MSE

In [36]:
# Use the overlap to get data from each data layer
demdtmoverdata = atlasmasked.data[overlapmask].reshape(overlapmask.shape)
atlasoverdata = demdtmmasked.data[overlapmask].reshape(overlapmask.shape)

In [37]:
delta = demdtmoverdata - atlasoverdata
delta2 = delta*delta

In [38]:
mse = delta2.mean()

In [39]:
mse

7.084522827927737